# Generate ELK Snapshot on Meas Node

### This notebook creates an ELK snapshot dir on the measurement node and prepares a .tar file to be exported

## Import

In [1]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

MFLib version  1.0.0 


In [2]:
import importlib
import mf_data_transfer
importlib.reload(mf_data_transfer)
from mf_data_transfer import mf_data_export

## Slice Info

In [3]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [4]:
mf = MFLib(slice_name)

Inititializing slice "MyMonitoredSlice" for MeasurementFramework.
Found meas node as meas-node at 2620:0:c80:1001:f816:3eff:fe94:a4d7
Bootstrap status indicates Slice Measurement Framework is ready.


In [5]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

-----------  ------------------------------------
Slice Name   MyMonitoredSlice
Slice ID     5dd2162f-26d1-460f-b06d-6833dc870378
Slice State  StableOK
Lease End    2023-06-01 20:16:29 +0000
-----------  ------------------------------------


## Create the data export object

In [6]:
mde = mf_data_export(slice_name=slice_name, meas_node_name=meas_node_name)

## Create ELK Snapshot

### Change directory permission

In [7]:
# This will change the permission of /var/lib/docker/volumes/elk_snapshotbackup/_data for elk to write snapshot to
mde.change_elk_dir_permission(node=mf.measurement_node_name)

### Create a snapshot repository

In [8]:
mde.create_snapshot_repo(repo_name="backup_repository")

{
  "acknowledged" : true
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   107  100    28  100    79    145    411 --:--:-- --:--:-- --:--:--   557
 

### Check the created repo

In [9]:
mde.verify_repo(repo_name="backup_repository")

{
  "backup_repository" : {
    "type" : "fs",
    "settings" : {
      "location" : "/usr/share/elasticsearch/backup"
    }
  }
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   131  100   131    0     0  32750      0 --:--:-- --:--:-- --:--:-- 32750
 

### List indice

In [10]:
mde.list_indices()

health status index                               uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   packetbeat-7.13.2-2023.05.31-000001 eG4a7pYZR5-aollsdxPofw   1   1       8213            0      7.5mb          3.6mb
green  open   .kibana_task_manager_7.13.0_001     _fnx9eXFTA-mnPvoUgnkVw   1   1         10          550    335.3kb        138.6kb
green  open   .apm-custom-link                    3Jc-EZltT3mTva-mmU89KA   1   1          0            0       416b           208b
green  open   .kibana-event-log-7.13.0-000001     t88RkrUuRO2iKb4OWw_zqA   1   1          1            0     11.2kb          5.6kb
green  open   metricbeat-7.13.2-2023.05.31-000001 OGb7GCOrR5uXivN3vUaCwA   1   1      60209            0     48.1mb           24mb
green  open   .apm-agent-configuration            ttpzBvquRG2Dyznx1AE1mw   1   1          0            0       416b           208b
green  open   .kibana_7.13.0_001                  Qpow41qHRL2Uik7Rf6EbQg   1   1   

### Create snapshot

In [11]:
# Default will include all indice. add arg (list) indice=[index1, index2] to specify indice
# May take a while
mde.create_elk_snapshot(repo_name="backup_repository", snapshot_name="snapshot_test")

{
  "snapshot" : {
    "snapshot" : "snapshot_test",
    "uuid" : "lHKG7MEDSliDw_U7Zrxv9w",
    "version_id" : 7130099,
    "version" : "7.13.0",
    "indices" : [
      ".apm-agent-configuration",
      ".kibana-event-log-7.13.0-000001",
      ".apm-custom-link",
      ".kibana_task_manager_7.13.0_001",
      "metricbeat-7.13.2-2023.05.31-000001",
      "filebeat-7.13.2-2023.05.31-000001",
      "packetbeat-7.13.2-2023.05.31-000001",
      ".ds-ilm-history-5-2023.05.31-000001",
      ".kibana_7.13.0_001",
      "kibana_sample_data_flights"
    ],
    "data_streams" : [
      "ilm-history-5"
    ],
    "include_global_state" : false,
    "state" : "SUCCESS",
    "start_time" : "2023-05-31T22:50:40.082Z",
    "start_time_in_millis" : 1685573440082,
    "end_time" : "2023-05-31T22:50:40.883Z",
    "end_time_in_millis" : 1685573440883,
    "duration_in_millis" : 801,
    "failures" : [ ],
    "shards" : {
      "total" : 10,
      "failed" : 0,
      "successful" : 10
    },
    "feature_

## Create dir for the elk tar file

In [12]:
elk_tar_dir = "/tmp/elk/tar"
mde.create_dir_for_elk_tar(dir_name=elk_tar_dir)

## Make the snapshot tar file

In [13]:
elk_tar_file_name = "elk_backup_0529"
mde.tar_elk_snapshot(dir_name=elk_tar_dir, file_name=elk_tar_file_name)

_data/
_data/meta-lHKG7MEDSliDw_U7Zrxv9w.dat
_data/snap-lHKG7MEDSliDw_U7Zrxv9w.dat
_data/indices/
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/meta-tvEBdIgB_jvqOm06lkos.dat
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__D_jHXFQAQ7S8kPPyhgHdhQ
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__GA6k8eS4QlWPlj3CWWfZwQ
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__TvfRBhiFRo-oOCApkcmmRQ
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__h39tRcDGQ2SQvUpNKiJEfw
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__GPU9-5-mTtOZSKzZtNlwvA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__DCdY01umTXu14LyjqOnNTA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__lr1VS5dkQ9KVPAhaYCwwnw
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__9sNgz2nSRoCd7SYhWjaCFA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__osxb5RWdS0KvMYCsWjedRA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__4tWLY9ZTQ3KzGGy7einfuA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__5RQ7S3g0R0qLhGkY4xhsJA
_data/indices/DSc-RTN6SUWVErJk2ZOVcA/0/__MFg1hesoSsSe

## Check the tar file

In [14]:
stdout, stderr= meas_node.execute("sudo ls -l --block-size=M /tmp/elk/tar")

total 41M
-rw-r--r-- 1 root root 41M May 31 22:50 elk_backup_0529.tar


### Now you have the elk snapshot .tar file you can use to restore data on another machine

In [15]:
# TODO: (MANUALLY DO FOR NOW)
# Move the tar file into the elk/files folder (or create new folder to place in this directory)


### Download index to local storage

Once the file is ready to be downloaded you can edit the code below to use the mf library.

In [16]:
# EDIT THIS - choose from "Files finished exporting above"
files = ["elk_backup_0529.tar"]

# Specify output location (Default is current directory)
outputDirectory = "./"
stdout = []
for file in files:
    stdout.append(mf.download_service_file("elk", "files/" + file , outputDirectory + file))

{'success': True, 'filename': './elk_backup_0529.tar', 'message': 'uploaded files/elk_backup_0529.tar successfully.'}


## Do **NOT** run the following if you want to restore the snapshot on measurement node since it can be achieved using a single curl command

### Optional: Delete the snapshot

In [ ]:
#mde.delete_snapshot(repo_name="backup_repository", snapshot_name="snapshot_test")

### Optional: Delete the repository

In [ ]:
#mde.delete_repo(repo_name="backup_repository")